# Multi-Agent System
## Imports

In [1]:
import os
from typing import Annotated, Any, Literal, TypedDict

from langchain_core.messages import (AIMessage, AnyMessage, HumanMessage,
                                     ToolMessage)
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessageGraph, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.types import Command

llm = ChatOllama(model='phi4-mini')
embedding = OllamaEmbeddings(model='nomic-embed-text')  

## 1. Supervisor Architecture

This architecture uses a central "supervisor" node to route tasks to specialized agents. It's a common and effective pattern for managing workflows where different agents have distinct capabilities.

### Key Concepts

- **State Management**: The `State` class inherits from `MessagesState` and adds a `result` field. This shared state is how agents pass information and results to each other.
- **Conditional Routing**: The `supervisor` node acts as a conditional router. It inspects the user's query and decides which agent (`math` or `research`) should handle it. This is a core concept in LangGraph for creating dynamic and intelligent workflows.
- **Nodes and Edges**: The graph is built by adding nodes (the supervisor and the agents) and edges (the connections between them). The `START` and `END` keywords are special nodes that define the beginning and end of the graph's execution.
- **Command Class**: The `Command` class is used to control the flow of the graph. It allows a node to specify which node to go to next (`goto`) and what data to update in the state (`update`).

### Execution Flow

1.  **Start**: The graph starts at the `supervisor` node.
2.  **Supervision**: The `supervisor` checks if a result is already present. If so, it routes to the `finish` node. Otherwise, it examines the last user message.
3.  **Routing**:
    *   If the message contains math-related keywords, it routes to the `math` agent.
    *   Otherwise, it routes to the `research` agent.
4.  **Agent Execution**: The selected agent (either `math` or `research`) processes the user's query and produces a result.
5.  **Return to Supervisor**: After execution, the agent routes back to the `supervisor`, updating the state with the `result`.
6.  **Finish**: The `supervisor` now sees a result in the state and routes to the `finish` node.
7.  **End**: The `finish` node formats the final output and terminates the graph execution.

In [2]:
class State(MessagesState):
    result: str


def last_user(state: State) -> str:
    for msg in reversed(state['messages']):
        if getattr(msg, 'type', getattr(msg, 'role', '')) in ('human', 'user'):
            return msg.content if hasattr(msg, 'content') else msg.get('content', '')
    return ''


def supervisor(state: State):
    if state.get('result'):
        return Command(goto='finish', update={})
    q = last_user(state).lower()
    if any(op in q for op in ['+', '-', '*', '/', 'solve', 'calc']):
        return Command(goto='math', update={})
    return Command(goto='research', update={})


def research_agent(state: State):
    q = last_user(state)
    resp = llm.invoke([HumanMessage(content=f'Answer briefly and factually: {q}')])
    return Command(goto='supervisor', update={'result': resp.content})


def math_agent(state: State):
    q = last_user(state)
    prompt = f'Solve the math expression or question. Reply with the final answer only:\n{q}'
    resp = llm.invoke([HumanMessage(content=prompt)])
    return Command(goto='supervisor', update={'result': resp.content.strip()})


def finish(state: State):
    return Command(goto=END, update={'messages': [AIMessage(content=state.get('result', 'No result'))]})


builder = StateGraph(State)
builder.add_node('supervisor', supervisor)
builder.add_node('research', research_agent)
builder.add_node('math', math_agent)
builder.add_node('finish', finish)
builder.add_edge(START, 'supervisor')
graph_supervisor = builder.compile()

# Demo usage (uncomment to run)
out = graph_supervisor.invoke({'messages': [HumanMessage("What's the capital of Japan?")]})
for m in out['messages']:
    m.pretty_print()
out = graph_supervisor.invoke({'messages': [HumanMessage('Compute 12*(3+4)')]})
for m in out['messages']:
    m.pretty_print()


================================ Human Message =================================

What's the capital of Japan?
================================== Ai Message ==================================

Tokyo
================================ Human Message =================================

Compute 12*(3+4)
================================== Ai Message ==================================

24


## 2. Network Architecture

This example demonstrates a network of collaborating agents where each agent can route to any other agent in the graph. This allows for more flexible and dynamic interactions compared to a strict hierarchy.

### Key Concepts

- **Peer-to-Peer Communication**: Unlike the supervisor model, agents in a network can communicate directly with each other. The `analyst_agent`, `researcher_agent`, and `strategist_agent` can all decide to pass control to one another.
- **State-Driven Routing**: Each agent's routing decision is based on the current state, specifically the content of the last message. For example, the `analyst_agent` routes to the `strategist_agent` if the message contains the word "strategy".
- **Looping and Iteration**: The agents can loop, passing control back and forth until a termination condition is met. In this case, the `iteration_count` in the `NetworkState` is used to prevent infinite loops.
- **Dynamic Endpoints**: The `__end__` literal in the `Command`'s type hint signifies that an agent can decide to end the graph's execution.

### Execution Flow

1.  **Entry Point**: The graph starts with the `analyst_agent`.
2.  **Analyst's Turn**: The `analyst_agent` processes the initial message and, based on its content, routes to either the `researcher_agent` or the `strategist_agent`.
3.  **Agent Collaboration**: The agents pass control to each other based on the logic within each agent function. For instance, the `researcher_agent` might find information and pass it to the `analyst_agent` for review.
4.  **Iteration Limit**: The `iteration_count` is incremented each time an agent is called. The graph will terminate if this count exceeds a certain threshold (in this case, 3 for most agents, 2 for the strategist).
5.  **Termination**: The graph ends when an agent routes to `__end__`, which happens either when the iteration limit is reached or when the `strategist_agent` has compiled a final recommendation.

In [3]:
class NetworkState(MessagesState):
    """State for network of collaborating agents"""

    iteration_count: int = 0


# Network Agents - each can route to any other
def analyst_agent(state: NetworkState) -> Command[Literal['researcher_agent', 'strategist_agent', '__end__']]:
    """Analyst agent that processes data and routes accordingly"""
    last_message = state['messages'][-1].content
    response = f"Analyst: I've reviewed '{last_message}'. The data suggests we need deeper investigation."
    new_state = {'messages': [AIMessage(content=response)], 'iteration_count': state.get('iteration_count', 0) + 1}
    if state.get('iteration_count', 0) >= 3:
        return Command(goto='__end__', update=new_state)
    elif 'strategy' in last_message.lower():
        return Command(goto='strategist_agent', update=new_state)
    else:
        return Command(goto='researcher_agent', update=new_state)


def researcher_agent(state: NetworkState) -> Command[Literal['analyst_agent', 'strategist_agent', '__end__']]:
    """Researcher agent that gathers information"""
    last_message = state['messages'][-1].content
    response = f"Researcher: Based on my investigation of '{last_message}', I found relevant background information."
    new_state = {'messages': [AIMessage(content=response)], 'iteration_count': state.get('iteration_count', 0) + 1}
    if state.get('iteration_count', 0) >= 3:
        return Command(goto='__end__', update=new_state)
    elif 'analysis' in last_message.lower():
        return Command(goto='analyst_agent', update=new_state)
    else:
        return Command(goto='strategist_agent', update=new_state)


def strategist_agent(state: NetworkState) -> Command[Literal['analyst_agent', 'researcher_agent', '__end__']]:
    """Strategist agent that develops plans"""
    last_message = state['messages'][-1].content
    response = (
        f"Strategist: For '{last_message}', I recommend a comprehensive approach combining multiple perspectives."
    )
    new_state = {'messages': [AIMessage(content=response)], 'iteration_count': state.get('iteration_count', 0) + 1}
    if state.get('iteration_count', 0) >= 2:
        final_response = 'Strategist: Final recommendation compiled. All agents have contributed to the solution.'
        return Command(goto='__end__', update={'messages': [AIMessage(content=final_response)]})
    else:
        return Command(goto='analyst_agent', update=new_state)


# Build the network graph
def create_network_graph():
    workflow = StateGraph(NetworkState)
    workflow.add_node('analyst_agent', analyst_agent)
    workflow.add_node('researcher_agent', researcher_agent)
    workflow.add_node('strategist_agent', strategist_agent)
    workflow.set_entry_point('analyst_agent')
    return workflow.compile()


# Usage example (uncomment to run)
# if __name__ == '__main__':
graph_network = create_network_graph()
result = graph_network.invoke(
    {'messages': [HumanMessage(content='We need to develop a strategy for market expansion')], 'iteration_count': 0}
)
print('Network Collaboration Result:')
for i, message in enumerate(result['messages']):
    print(f'{i + 1}. {message.content}')
    print()


Network Collaboration Result:
1. We need to develop a strategy for market expansion

2. Analyst: I've reviewed 'We need to develop a strategy for market expansion'. The data suggests we need deeper investigation.

3. Strategist: For 'Analyst: I've reviewed 'We need to develop a strategy for market expansion'. The data suggests we need deeper investigation.', I recommend a comprehensive approach combining multiple perspectives.

4. Analyst: I've reviewed 'Strategist: For 'Analyst: I've reviewed 'We need to develop a strategy for market expansion'. The data suggests we need deeper investigation.', I recommend a comprehensive approach combining multiple perspectives.'. The data suggests we need deeper investigation.

5. Strategist: Final recommendation compiled. All agents have contributed to the solution.



## 3. Hierarchical Architecture

This architecture organizes agents into teams, or subgraphs, which are then orchestrated by a top-level supervisor. This is useful for breaking down complex problems into smaller, manageable parts.

### Key Concepts

- **Subgraphs**: The `research_team` and `analysis_team` are themselves complete LangGraphs. These subgraphs are then used as nodes in the main graph. This allows for modular and reusable components.
- **Hierarchical Control**: The top-level `supervisor` decides which team (subgraph) to delegate the task to. This is a powerful way to manage complexity, as the supervisor only needs to know about the teams, not the individual agents within them.
- **State Propagation**: The state (`TopState`) is passed down from the main graph to the subgraphs. This means that the agents within the subgraphs have access to the same information as the top-level supervisor.
- **Encapsulation**: Each subgraph is self-contained. The `research_team` has its own internal logic (`gather` and `synth`), and the `analysis_team` has its own (`compute` and `summary`). This makes the overall system easier to understand and maintain.

### Execution Flow

1.  **Top-Level Supervision**: The graph starts at the top-level `supervisor`.
2.  **Delegation**: The supervisor inspects the user's query and routes the task to either the `research_team` or the `analysis_team`.
3.  **Subgraph Execution**: The selected subgraph executes its internal workflow. For example, if the `research_team` is chosen, it will first `gather` data and then `synthesize` it.
4.  **Return to Main Graph**: Once the subgraph has finished, it returns control to the main graph.
5.  **Final Node**: The main graph then routes to the `final` node, which formats the output from the `notes` field in the state.
6.  **End**: The graph execution terminates.

In [4]:
class TopState(MessagesState):
    data: str
    notes: str


def last_user(s: TopState) -> str:
    for m in reversed(s['messages']):
        if getattr(m, 'type', getattr(m, 'role', '')) in ('human', 'user'):
            return m.content if hasattr(m, 'content') else m.get('content', '')
    return ''


DOCS = [
    'Python was created by Guido van Rossum.',
    'The capital of Japan is Tokyo.',
    'LangGraph helps build multi-agent workflows.',
]
V_DOCS = [embedding.embed_query(d) for d in DOCS]


def gather(state: TopState):
    q = last_user(state)
    qv = embedding.embed_query(q)
    idx = max(range(len(DOCS)), key=lambda i: sum(a * b for a, b in zip(V_DOCS[i], qv)))
    return {'data': DOCS[idx]}


def synthesize(state: TopState):
    prompt = f'Using this note, answer the user briefly:\nNote: {state.get("data", "")}\nUser: {last_user(state)}'
    resp = llm.invoke([HumanMessage(content=prompt)])
    return {'notes': resp.content}


def compute(state: TopState):
    q = last_user(state)
    resp = llm.invoke([HumanMessage(content=f'Analyze and give a score 0-1 with a reason for: {q}')])
    return {'data': resp.content}


def summarize(state: TopState):
    resp = llm.invoke([HumanMessage(content=f'Summarize to one sentence: {state.get("data", "")}')])
    return {'notes': resp.content}


# Research subgraph
r = StateGraph(TopState)
r.add_node('gather', gather)
r.add_node('synth', synthesize)
r.add_edge(START, 'gather')
r.add_edge('gather', 'synth')
r.add_edge('synth', END)
research_team = r.compile()

# Analysis subgraph
a = StateGraph(TopState)
a.add_node('compute', compute)
a.add_node('summary', summarize)
a.add_edge(START, 'compute')
a.add_edge('compute', 'summary')
a.add_edge('summary', END)
analysis_team = a.compile()


def supervisor(state: TopState):
    q = last_user(state).lower()
    if state.get('notes'):
        return Command(goto='final', update={})
    if any(op in q for op in ['+', '-', '*', '/', 'calc', 'solve']):
        return Command(goto='analysis_team', update={})
    return Command(goto='research_team', update={})


def final_node(state: TopState):
    return Command(goto=END, update={'messages': [AIMessage(content=state.get('notes', 'No notes'))]})


b = StateGraph(TopState)
b.add_node('supervisor', supervisor)
b.add_node('research_team', research_team)
b.add_node('analysis_team', analysis_team)
b.add_node('final', final_node)
b.add_edge(START, 'supervisor')
b.add_edge('research_team', 'final')
b.add_edge('analysis_team', 'final')
graph_hierarchical = b.compile()

# Demo usage (uncomment to run)
out = graph_hierarchical.invoke({'messages': [HumanMessage('Who created Python?')]})
for m in out['messages']:
    m.pretty_print()
out = graph_hierarchical.invoke({'messages': [HumanMessage('Quickly estimate 20*(5-2).')]})
for m in out['messages']:
    m.pretty_print()


================================ Human Message =================================

Who created Python?
================================== Ai Message ==================================

Python was created by Guido van Rossum.
================================ Human Message =================================

Quickly estimate 20*(5-2).
================================== Ai Message ==================================

I will quickly analyze the expression `20 * (5 - 2)` by first calculating inside the parentheses `(5 - 2) = 3` and then multiplying to get `20 * 3 = 60`. This straightforward arithmetic yields an accuracy score of `1 (100%)`, as there are no errors or uncertainties involved.
